# Datos de diciembre

In [1]:
import pandas as pd
dic_clientes = pd.read_csv("clientes_diciembre.csv", sep = "|")
dic_consumos = pd.read_csv("consumos_diciembre.csv", sep = "|")
dic_fin = pd.read_csv("financiacion_diciembre.csv", sep = "|")
dic_prod = pd.read_csv("productos_diciembre.csv", sep = "|")

In [2]:
print('Tamaño de tabla de consumos diciembre:',dic_consumos.shape[0])
print('Tamaño de tabla de clientes diciembre:',dic_clientes.shape[0])
print('Tamaño de tabla de financiación diciembre:',dic_fin.shape[0])
print('Tamaño de tabla de productos diciembre:',dic_prod.shape[0])

Tamaño de tabla de consumos diciembre: 95467
Tamaño de tabla de clientes diciembre: 95467
Tamaño de tabla de financiación diciembre: 24198
Tamaño de tabla de productos diciembre: 95467


Miro si los ids son los mismos para contatenar las tablas

In [3]:
print('Clientes en tabla financiacion\n', dic_clientes['id'].isin(dic_fin['id']).value_counts())
print('Clientes en tabla consumos\n',dic_clientes['id'].isin(dic_consumos['id']).value_counts())
print('Clientes en tabla productos\n',dic_clientes['id'].isin(dic_prod['id']).value_counts())

Clientes en tabla financiacion
 False    71269
True     24198
Name: id, dtype: int64
Clientes en tabla consumos
 True    95467
Name: id, dtype: int64
Clientes en tabla productos
 True    95467
Name: id, dtype: int64


In [4]:
dic_cl_con = pd.merge(dic_clientes, dic_consumos, on = "id", how = "left")
dic_cl_con_f = pd.merge(dic_cl_con, dic_fin, on= "id", how = "left")
dic_full = pd.merge(dic_cl_con_f, dic_prod, on = "id", how = "left")
del dic_cl_con
del dic_cl_con_f

# Datos de enero

In [5]:
en_clientes = pd.read_csv("clientes_enero.csv", sep = "|", parse_dates = ["antiguedad"])
en_consumos = pd.read_csv("consumos_enero.csv", sep = "|")
en_fin = pd.read_csv("financiacion_enero.csv", sep = "|")
en_prod = pd.read_csv("productos_enero.csv", sep = "|")

In [6]:
print('Tamaño de tabla de consumos enero:',en_consumos.shape[0])
print('Tamaño de tabla de clientes enero:',en_clientes.shape[0])
print('Tamaño de tabla de financiación enero:',en_fin.shape[0])
print('Tamaño de tabla de productos enero:',en_prod.shape[0])

Tamaño de tabla de consumos enero: 92711
Tamaño de tabla de clientes enero: 92711
Tamaño de tabla de financiación enero: 25332
Tamaño de tabla de productos enero: 92711


In [7]:
print('Clientes en tabla financiacion\n', en_clientes['id'].isin(en_fin['id']).value_counts())
print('Clientes en tabla consumos\n',en_clientes['id'].isin(en_consumos['id']).value_counts())
print('Clientes en tabla productos\n',en_clientes['id'].isin(en_prod['id']).value_counts())

Clientes en tabla financiacion
 False    67379
True     25332
Name: id, dtype: int64
Clientes en tabla consumos
 True    92711
Name: id, dtype: int64
Clientes en tabla productos
 True    92711
Name: id, dtype: int64


In [8]:
en_cl_con = pd.merge(en_clientes, en_consumos, on = "id", how = "left")
en_cl_con_f = pd.merge(en_cl_con, en_fin, on = "id", how = "left")
en_full = pd.merge(en_cl_con_f, en_prod, on = "id", how = "left")
del en_cl_con
del en_cl_con_f

# Clientes perdidos

In [9]:
print('Clientes en Enero: ',len(en_full['id'].unique()))
print('Clientes en Diciembre: ',len(dic_full['id'].unique()))
print('Se han ido: ', len(dic_full) - dic_full.id.isin(en_full.id).sum(), ' clientes')
print('En enero hay ', len(dic_full['id'].unique()) - len(en_full['id'].unique()), 'clientes menos que en diciembre')

Clientes en Enero:  92711
Clientes en Diciembre:  95467
Se han ido:  7085  clientes
En enero hay  2756 clientes menos que en diciembre


# Variable objetivo

Creo la variable objetivo, 0 si el cliente sigue en la empresa, 1 si el cliente se ha ido.

In [10]:
import numpy as np
churn = pd.DataFrame()
churn['churn'] = np.where(dic_full.id.isin(en_full.id) == True, 0, 1)
churn['id'] =dic_full['id'] 

# Antiguedad del cliente

Creo una nueva variable 'duracion', que es el tiempo que lleva el cliente en la empresa.

In [11]:
dic_full['antiguedad'] = pd.to_datetime(dic_full['antiguedad'])
en_full['antiguedad'] = pd.to_datetime(dic_full['antiguedad'])

In [12]:
from datetime import datetime
dic_full['duracion'] = (datetime(2019,12,31) - dic_full["antiguedad"])
en_full['duracion'] = (datetime(2020,1,31) - en_full["antiguedad"])

In [13]:
dic_full.head()

,id,edad,facturacion,antiguedad,provincia,num_lineas,num_dt,incidencia,num_llamad_ent,num_llamad_sal,mb_datos,seg_llamad_ent,seg_llamad_sal,financiacion,imp_financ,descuentos,conexion,vel_conexion,TV,duracion
0,1,63,216.028109,2018-11-23 08:48:00,La Rioja,5,NaN,NaN,110,79,10897,12806,13751,NaN,NaN,NaN,FIBRA,50MB,tv-futbol,402 days 15:12:00
1,2,84,255.830842,2017-08-22 03:19:00,Vizcaya,3,NaN,NaN,189,89,18657,6499,10862,NaN,NaN,SI,FIBRA,600MB,tv-futbol,860 days 20:41:00
2,3,66,135.768153,2001-12-27 13:50:00,Albacete,4,NaN,NaN,129,30,15511,17013,16743,NaN,NaN,SI,ADSL,35MB,tv-futbol,6577 days 10:10:00
3,4,69,255.658527,2015-08-08 10:53:00,Lugo,4,NaN,NaN,51,52,12670,3393,6771,NaN,NaN,NaN,FIBRA,200MB,tv-familiar,1605 days 13:07:00
4,5,30,22.302845,1997-08-29 02:19:00,Tarragona,2,2.0,NaN,183,3,23756,18436,4485,NaN,NaN,NaN,ADSL,10MB,tv-futbol,8158 days 21:41:00


# Diciembre y Enero

In [14]:
dic_full["mes"] = "dic"
en_full["mes"] = "en"

In [15]:
data = pd.concat([dic_full, en_full], sort = False, ignore_index=True)
data.shape

(188178, 21)

In [16]:
data.head(5)

,id,edad,facturacion,antiguedad,provincia,num_lineas,num_dt,incidencia,num_llamad_ent,num_llamad_sal,...,seg_llamad_ent,seg_llamad_sal,financiacion,imp_financ,descuentos,conexion,vel_conexion,TV,duracion,mes
0,1,63,216.028109,2018-11-23 08:48:00,La Rioja,5,NaN,NaN,110,79,...,12806,13751,NaN,NaN,NaN,FIBRA,50MB,tv-futbol,402 days 15:12:00,dic
1,2,84,255.830842,2017-08-22 03:19:00,Vizcaya,3,NaN,NaN,189,89,...,6499,10862,NaN,NaN,SI,FIBRA,600MB,tv-futbol,860 days 20:41:00,dic
2,3,66,135.768153,2001-12-27 13:50:00,Albacete,4,NaN,NaN,129,30,...,17013,16743,NaN,NaN,SI,ADSL,35MB,tv-futbol,6577 days 10:10:00,dic
3,4,69,255.658527,2015-08-08 10:53:00,Lugo,4,NaN,NaN,51,52,...,3393,6771,NaN,NaN,NaN,FIBRA,200MB,tv-familiar,1605 days 13:07:00,dic
4,5,30,22.302845,1997-08-29 02:19:00,Tarragona,2,2.0,NaN,183,3,...,18436,4485,NaN,NaN,NaN,ADSL,10MB,tv-futbol,8158 days 21:41:00,dic


# Transformación de variables